In [1]:
import cv2
import typing
import numpy as np
import os

from mltu.inferenceModel import OnnxInferenceModel
from mltu.utils.text_utils import ctc_decoder, get_cer
import matplotlib.pyplot as plt


In [2]:
class ImageToWordModel(OnnxInferenceModel):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def predict(self, image: np.ndarray):
        image = cv2.resize(image, self.input_shape[:2][::-1])

        image_pred = np.expand_dims(image, axis=0).astype(np.float32)

        preds = self.model.run(None, {self.input_name: image_pred})[0]

        text = ctc_decoder(preds, self.vocab)[0]

        return text

In [3]:
# salvar imagem com sua label e predição

def save_image(image_path, label, prediction_text, distance):
    image = cv2.imread(image_path)
    plt.imshow(image)
    plt.title(f"Label: {label}, Prediction: {prediction_text}, Distance: {distance}")
    if not os.path.isfile(f"../data/imagens/{image_path.split('/')[-1]}"): 
        plt.savefig(f"../data/imagens/{image_path.split('/')[-1]}")
    else:
        print("existe")    

In [ ]:
if __name__ == "__main__":
    import pandas as pd
    from tqdm import tqdm

    model = ImageToWordModel(
        model_path="Models/08_handwriting_recognition_torch/202306131539/model.onnx")

    df = pd.read_csv(
        "Models/08_handwriting_recognition_torch/202306131539/val.csv").values.tolist()

    accum_cer = []
    for image_path, label in tqdm(df):
        image = cv2.imread(image_path)

        prediction_text = model.predict(image)

        cer, distance = get_cer(prediction_text, label)
        print(
            f"Image: {image_path}, Label: {label}, Prediction: {prediction_text}, CER: {cer}, Distance: {distance} ")
        
        save_image(image_path, label, prediction_text, distance)

        accum_cer.append(cer)

    print(f"Average CER: {np.average(accum_cer)}")